In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.metrics import accuracy_score , classification_report
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import re
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
! pip install kaggle

Upload you kaggle.json file

In [ ]:
%mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


Importing Twitter sentiment dataset

In [ ]:
1.#Api to fetch the daatset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 93% 75.0M/80.9M [00:01<00:00, 72.4MB/s]
100% 80.9M/80.9M [00:01<00:00, 68.1MB/s]


In [ ]:
#extracting the compressed dataset
from zipfile import ZipFile
df = '/content/sentiment140.zip'

with ZipFile(df , 'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [ ]:
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv' , encoding = 'latin-1')

In [ ]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
#naming the columns
columns_name = ['target' , 'id' , 'date' , 'flag' , 'user' , 'text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv' ,names = columns_name, encoding = 'latin-1')


In [ ]:
twitter_data.shape

(1600000, 6)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
twitter_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   id      1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   flag    1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [ ]:
#Handling missing values
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [ ]:
#Distrinution of target variable
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [ ]:
#Replacing the value 4 to 1
twitter_data.replace({'target' : {4:1}} , inplace = True)


In [ ]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0 ---->>> NEGATIVE TWEET

1 ---->>> POSITIVE TWEET

Data is distributed equally .

**TOKENIZATION**

In [ ]:
#It will create a list of sentences
from nltk.tokenize import sent_tokenize
twitter_data['sentence'] = twitter_data['text'].apply(sent_tokenize)

In [ ]:
#It will create a list of words from sentences
from nltk.tokenize import wordpunct_tokenize
twitter_data['words'] = twitter_data['text'].apply(wordpunct_tokenize)

Summary: Created 2 new columns ['sentence' , 'words']

Step 1: Converted paragraphs to sentences.

step 2: Converted seentences to words.

**Removing StopWords**

In [ ]:
#Creating a function to remove stopwords
def remove_stopwords(token):
  stop_words = set(stopwords.words('english'))
  filtered_token = [i for i in token if i not in stop_words]
  return filtered_token

#Applying the function in words column
twitter_data['filtered_words'] = twitter_data['words'].apply(remove_stopwords)

**Stemming**

Stemming is the process of converting words to root words

Examples --> {'ACTOR' , 'ACTRESS' , 'ACTING'}---> 'ACT'

In [ ]:
from nltk.stem import SnowballStemmer
stem = SnowballStemmer('english')

def stem_word(token):
  return [stem.stem(i) for i in token]

twitter_data['filtered_text'] = twitter_data['filtered_words'].apply(stem_word)

**Removing punctiations and special characters**

In [ ]:
#Function to remove special characters and punctuations
def clean_tokens(tokens):
    # Regular expression to match non-alphanumeric characters
    pattern = re.compile(r'[^a-zA-Z0-9]')
    # Apply the pattern to each token
    cleaned_tokens = [pattern.sub('', token) for token in tokens]
    # Remove empty strings that may have resulted from the cleaning process
    cleaned_tokens = [token for token in cleaned_tokens if token]
    return cleaned_tokens

#Applying the function in new column
twitter_data['filtered_text_clean'] = twitter_data['filtered_text'].apply(clean_tokens)

In [ ]:
twitter_data['filtered_text_clean'].head(3)

0    [switchfoot, http, twitpic, com, 2y1zl, awww, ...
1    [upset, updat, facebook, text, might, cri, res...
2    [kenichan, i, dive, mani, time, ball, manag, s...
Name: filtered_text_clean, dtype: object

In [ ]:
# Convert the list of words back to a string
twitter_data['filtered_text_str'] = twitter_data['filtered_text_clean'].apply(lambda x: ' '.join(x))

In [ ]:
twitter_data.head(2)

,target,id,date,flag,user,text,sentence,words,filtered_words,filtered_text,filtered_text_clean,filtered_text_str
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","[@switchfoot http://twitpic.com/2y1zl - Awww, ...","[@, switchfoot, http, ://, twitpic, ., com, /,...","[@, switchfoot, http, ://, twitpic, ., com, /,...","[@, switchfoot, http, ://, twitpic, ., com, /,...","[switchfoot, http, twitpic, com, 2y1zl, awww, ...",switchfoot http twitpic com 2y1zl awww bummer ...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,[is upset that he can't update his Facebook by...,"[is, upset, that, he, can, ', t, update, his, ...","[upset, ', update, Facebook, texting, ..., mig...","[upset, ', updat, facebook, text, ..., might, ...","[upset, updat, facebook, text, might, cri, res...",upset updat facebook text might cri result sch...


Our data is clean . Now we can convert the text to vectors

**Splitting the data**

In [ ]:
x = twitter_data['filtered_text_str']
y = twitter_data['target']

#Splitting the data
x_train , x_test , y_train , y_test = train_test_split(x , y , test_size = 0.3 , random_state = 42)

**TF/IDF**

In [ ]:
#vectorize text data
vector = TfidfVectorizer(max_features = 10000)
x_train_vec = vector.fit_transform(x_train)
x_test_vec = vector.transform(x_test)


**Building Model**

**MODEL 1: LOGISTIC REGRESSION**

In [ ]:
#Using Logistic Regression
model = LogisticRegression()
model.fit(x_train_vec , y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
#predict
y_pred_test = model.predict(x_test_vec)
y_pred_train = model.predict(x_train_vec)


#Evaluating metrics
#Training data
accuracy_train = accuracy_score(y_train , y_pred_train)
print('Accuracy Train:' , accuracy_train)

accuracy_test = accuracy_score(y_pred_test , y_test)
print('Accuracy score Test:' , accuracy_test)

classification = classification_report(y_pred_test , y_test)
print('classification report:')
print(classification)

Accuracy Train: 0.7803517857142858
Accuracy score Test: 0.7757375
classification report:
              precision    recall  f1-score   support

           0       0.76      0.79      0.77    230205
           1       0.80      0.77      0.78    249795

    accuracy                           0.78    480000
   macro avg       0.78      0.78      0.78    480000
weighted avg       0.78      0.78      0.78    480000



**Model 2: Naive Bayes**

In [ ]:
from sklearn.naive_bayes import MultinomialNB
# Initialize the MultinomialNB classifier
model2 = MultinomialNB()

# Train the model
model2.fit(x_train_vec, y_train)

# Predict on the test data
y_predtest_2 = model2.predict(x_test_vec)
# Predict on the training data
y_predtrain_2 = model2.predict(x_train_vec)


#Evaluating metrics
#Training data
accuracy_train2 = accuracy_score(y_train , y_predtrain_2)
print('Accuracy Train:' , accuracy_train2)

accuracy_test2 = accuracy_score(y_test , y_predtest_2)
print('Accuracy score Test:' , accuracy_test2)

classification2 = classification_report(y_test , y_predtest_2)
print('classification report:')
print(classification2)

Accuracy Train: 0.7626517857142857
Accuracy score Test: 0.7580083333333333
classification report:
              precision    recall  f1-score   support

           0       0.75      0.76      0.76    239361
           1       0.76      0.75      0.76    240639

    accuracy                           0.76    480000
   macro avg       0.76      0.76      0.76    480000
weighted avg       0.76      0.76      0.76    480000

